# Checked Eopatch based on Geopackage

## Esse notebook serve para validar se os dados dos Eopatches em disco tem a mesma quantidade dos dados dentro do Geopackage

In [1]:
# Load the auto reload extension to automatically reload modules when files on disk are updated
%load_ext autoreload
# it will automatically be reloaded without the need to restart the kernel.
%autoreload 2

### Imports

In [2]:
import boto3
import geopandas as gpd
from io import BytesIO
import pandas as pd
from folium import GeoJson
import folium
import numpy as np
from tqdm import tqdm
from glob import glob
#import local modules
import sys
sys.path.append("/agrilearn_app/agrilearn/")
from agrilearn.utils import s3_utils, str_utils, eopatch_utils

ImportError: cannot import name 's3_utils' from 'agrilearn.utils' (unknown location)

### Global Variables

In [3]:
DATASET_PATH_LOCAL =  "/agrilearn_app/datasets/teste_pre_safra_2024_2025/geopackage/processed/data_merged_crop_120_06_08_2024_input_mvp_processing.gpkg"

# 1. Read Geopackage from Local Disck

## 1.1 Open Dataset

In [4]:
df = gpd.read_file(DATASET_PATH_LOCAL)

In [5]:
df.head()

,interest_area_id,period,start_season,no_till_farming,emergence_date,emergence_sensor,gt_class,senescence_date,senescence_sensor,harvest_date,...,start_of_season,end_of_season,peaks,length_of_season,progress_status,uf,uuid,peak_date,dataset_part,geometry
0,0,2023/2024,2023-10-01,None,2024-01-12,MSI / Sentinel 2,SOYBEAN,2024-03-04,MSI / Sentinel 2,None,...,NaT,NaT,None,None,None,None,None,None,test,"MULTIPOLYGON (((-49.87426 -17.49841, -49.87428..."
1,1,2023/2024,2023-10-01,None,2024-01-12,MSI / Sentinel 2,SOYBEAN,2024-03-04,MSI / Sentinel 2,None,...,NaT,NaT,None,None,None,None,None,None,test,"MULTIPOLYGON (((-49.87649 -17.50258, -49.88479..."
2,2,2023/2024,2023-10-01,None,2023-12-06,OLI / Landsat 9,SOYBEAN,2024-02-24,OLI / Landsat 9,2024-03-05,...,NaT,NaT,None,None,None,None,None,None,test,"MULTIPOLYGON (((-48.5898 -4.8188, -48.58982 -4..."
3,3,2023/2024,2023-10-01,None,2024-01-17,OLI / Landsat 8,SOYBEAN,2024-03-14,MSI / Sentinel 2,None,...,NaT,NaT,None,None,None,None,None,None,test,"MULTIPOLYGON (((-47.92268 -17.87173, -47.92267..."
4,4,2023/2024,2023-10-01,None,2024-01-17,OLI / Landsat 8,SOYBEAN,2024-03-13,OLI / Landsat 9,None,...,NaT,NaT,None,None,None,None,None,None,test,"MULTIPOLYGON (((-47.92267 -17.87151, -47.92268..."


In [6]:
df['dataset_part'].value_counts(normalize=True)*100

dataset_part
test    100.0
Name: proportion, dtype: float64

In [7]:
report_geopackage = df.groupby(['gt_class', 'dataset_part'], as_index=False)\
                            .agg(count=('state','count'))\
                            .sort_values(['gt_class', 'count'], ascending=[False, False]).reset_index(drop=True)
report_geopackage

,gt_class,dataset_part,count
0,WHEAT,test,50
1,SUGAR_CANE,test,50
2,SOYBEAN,test,7283
3,RICE,test,81
4,PASTURE,test,50
5,COTTON,test,267
6,CORN,test,579


# 2. Read eopatch files from Local disk

## 2.1 Open files

In [8]:
OUTPUT_DATASETS_PATH_LOCAL = "/agrilearn_app/datasets/teste_pre_safra_2024_2025/eopatch/processed/**"
df_eopatch_files = eopatch_utils.get_local_path_from_eopatch(OUTPUT_DATASETS_PATH_LOCAL,
                                                             new_label='path')

In [9]:
df_eopatch_files['path'].shape

(12864,)

In [10]:
df_eopatch_files.head()

,path
0,/agrilearn_app/datasets/teste_pre_safra_2024_2...
1,/agrilearn_app/datasets/teste_pre_safra_2024_2...
2,/agrilearn_app/datasets/teste_pre_safra_2024_2...
3,/agrilearn_app/datasets/teste_pre_safra_2024_2...
4,/agrilearn_app/datasets/teste_pre_safra_2024_2...


## 2.2 Create new Features

In [26]:
df_eopatch_files['dataset_part'] = 'test' #str_utils.get_series_from_string_list(df_eopatch_files['path'], categories=['train', 'val', 'test'])
df_eopatch_files['monitoring_class'] = str_utils.get_series_from_string_list(
    df_eopatch_files['path'], categories=["CORN", "SOYBEAN", "COTTON", "RICE", "SUGAR_CANE", "PASTURE", "WHEAT"])

In [27]:
report_geopackage

,gt_class,dataset_part,count
0,WHEAT,test,50
1,SUGAR_CANE,test,50
2,SOYBEAN,test,7283
3,RICE,test,81
4,PASTURE,test,50
5,COTTON,test,267
6,CORN,test,579


In [28]:
df_eopatch_files['monitoring_class'].value_counts()

monitoring_class
SOYBEAN       9760
CORN          2701
COTTON         232
RICE            81
WHEAT           50
SUGAR_CANE      40
Name: count, dtype: int64

In [34]:
df_eopatch_files['path'].iloc[0]

'/agrilearn_app/datasets/teste_pre_safra_2024_2025/eopatch/processed/start_2017-09-01_end_2018-06-01_monitoring_class_RICE_epsg4326_minxymaxxy_-53dot295526525023355_-23dot223792263775564_-53dot27749190858382_-23dot210148771468873_7_20/eopatch_0_col-0_row-0'

## 2.3 Report Generation

In [31]:
report_eopatch = df_eopatch_files.groupby(['monitoring_class', 'dataset_part'], as_index=False)\
                                        .agg(count=('path','count'))\
                                        .sort_values(['monitoring_class', 'count'], ascending=[False, False]).reset_index(drop=True)
report_eopatch

,monitoring_class,dataset_part,count
0,WHEAT,test,50
1,SUGAR_CANE,test,40
2,SOYBEAN,test,9760
3,RICE,test,81
4,COTTON,test,232
5,CORN,test,2701


# 3. Compare files between Geopackage and Eopatch

In [32]:
pd.concat([report_eopatch, report_geopackage])

,monitoring_class,dataset_part,count,gt_class
0,WHEAT,test,50,NaN
1,SUGAR_CANE,test,40,NaN
2,SOYBEAN,test,9760,NaN
3,RICE,test,81,NaN
4,COTTON,test,232,NaN
5,CORN,test,2701,NaN
0,NaN,test,50,WHEAT
1,NaN,test,50,SUGAR_CANE
2,NaN,test,7283,SOYBEAN
3,NaN,test,81,RICE


In [20]:
print('Se True então os reports são iguais e todas as imagens foram baixadas')

Se True então os reports são iguais e todas as imagens foram baixadas


In [21]:
report_eopatch.equals(report_geopackage)

False